In [ ]:
エクアドルにある Favorita 店舗で販売されている何千もの製品フの売上を予測します。 トレーニング データには、日付、店舗と製品の情報、その商品がプロモーションされているかどうか、販売数が含まれます。 追加ファイルには、モデルの構築に役立つ補足情報が含まれています。

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Jupyter Notebook環境でのマジックコマン.グラフやプロットがNotebook上で直接表示。
%matplotlib inline
import warnings
# プログラムが実行される際に発生する警告メッセージを無視
warnings.filterwarnings('ignore')

# データの読み込み

In [5]:
df_train = pd.read_csv('data/train.csv',usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float64',},parse_dates=['date'],infer_datetime_format=True)

df_test = pd.read_csv('data/test.csv')
df_stores = pd.read_csv('data/stores.csv')
df_oil = pd.read_csv('data/oil.csv')
df_holidays = pd.read_csv('data/holidays_events.csv')
df_transactions = pd.read_csv('data/transactions.csv')

# データの分析

In [7]:
print("trainデータ")
print(df_train.info())

trainデータ
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   store_nbr    category      
 2   family       category      
 3   sales        float64       
 4   onpromotion  int64         
dtypes: category(2), datetime64[ns](1), float64(1), int64(1)
memory usage: 74.4 MB
None
